In [1]:
'''
attempt to predict housing price
light feature engineering
one_hot encoding
standardize feature

try different moodels for regression
'''

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

from category_encoders import TargetEncoder

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 200)

In [2]:
df = pd.read_csv('C:\\Users\\xzhao\\Data\\houseprice.csv')

In [3]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [4]:
mdf = pd.DataFrame()

In [5]:
# this changes the type of price, making it unfit for model
# df['price'] = df['price'].apply(lambda x: "{:,.2f}".format(x))

In [6]:
df = df[df['price'] != 0]

In [7]:
# df = df[df['price']< 1000000]

In [8]:
# ## Anommaly detection to find weired price
# ## concerned about using it without domain knowledge, too much assumption made inexplicitly

# from sklearn.ensemble import IsolationForest
# import numpy as np

# weiredprice = df['price'].values.reshape(-1,1)

# clf = IsolationForest(random_state=0)
# clf.fit(weiredprice)

# y_hat = pd.DataFrame(clf.predict(weiredprice))

# df[y_hat[0]==-1]

In [9]:
def understand(feature,bool_param):
    
#     print(df[feature].unique())
#     print('empty records:', df[df[feature].isna()])
#     print('data description:', df[feature].describe())

    if bool_param:
        mdf[feature] = df[feature]

In [10]:
feature = 'price'
understand(feature,True)

# hand pick and corret targets

In [11]:
df[['price','bedrooms','bathrooms','sqft_living','sqft_lot']][mdf["price"]>4000000]

,price,bedrooms,bathrooms,sqft_living,sqft_lot
2286,7062500.0,5.0,4.50,10040,37325
2654,4668000.0,5.0,6.75,9640,13068
2761,4489000.0,4.0,3.00,6430,27517
4346,12899000.0,3.0,2.50,2190,11394
4350,26590000.0,3.0,2.00,1180,7793


In [12]:
df[(df['bedrooms']==3) & (df['bathrooms']==2) & (df['sqft_living']>500) & (df['sqft_lot']>100)]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
2,2014-05-02 00:00:00,342000.000000,3.0,2.0,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
38,2014-05-02 00:00:00,403000.000000,3.0,2.0,1960,13100,1.0,0,2,5,1650,310,1957,0,17825 4th Ave SW,Normandy Park,WA 98166,USA
52,2014-05-02 00:00:00,805000.000000,3.0,2.0,2710,4500,1.5,0,0,4,1880,830,1929,0,6222 30th Ave NE,Seattle,WA 98115,USA
93,2014-05-05 00:00:00,770000.000000,3.0,2.0,2350,5700,1.5,0,0,4,1810,540,1939,1989,2171 Boyer Ave E,Seattle,WA 98112,USA
112,2014-05-05 00:00:00,300000.000000,3.0,2.0,1050,10072,1.0,0,0,3,1050,0,1986,0,27106 NE Miller St,Duvall,WA 98019,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,2014-05-28 00:00:00,450385.714286,3.0,2.0,1300,3731,1.0,0,0,3,900,400,1993,0,7250 Rainier Ave S,Seattle,WA 98118,USA
4439,2014-05-30 00:00:00,237481.250000,3.0,2.0,2120,7560,1.0,0,0,3,2120,0,1991,0,18600 135th Pl SE,Renton,WA 98058,USA
4461,2014-06-05 00:00:00,420642.857143,3.0,2.0,1330,8100,1.0,0,0,4,1330,0,1969,0,13817 125th Ave NE,Kirkland,WA 98034,USA
4493,2014-06-13 00:00:00,300000.000000,3.0,2.0,2020,8555,1.0,0,0,4,1220,800,1957,2001,13545 39th Ave NE,Seattle,WA 98125,USA


In [13]:
mdf["price"].iloc[4350] = 265900
mdf["price"].iloc[4346] = 1289900
mdf["price"].iloc[2286] = 706250

In [14]:
# mdf = mdf[~mdf['price'].isin([265900,1289900,706250])]

# scrub and engineer feature

In [15]:
feature = 'date'
understand(feature,False)

In [16]:
mdf['date'] = pd.to_datetime(df['date'])
mdf['month'] = mdf['date'].dt.month
# mdf['day'] = mdf['date'].dt.day
mdf.drop('date', axis=1, inplace=True)
# mdf.drop('day', axis=1, inplace=True)

In [17]:
mdf['price'] = mdf['price'].astype(int)

In [18]:
feature = 'bedrooms'
understand(feature,True)

In [19]:
feature = 'bathrooms'
understand(feature,True)

In [20]:
df[df['bedrooms']==0]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
2365,2014-06-12 00:00:00,1095000.0,0.0,0.0,3064,4764,3.5,0,2,3,3064,0,1990,2009,814 E Howe St,Seattle,WA 98102,USA
3209,2014-06-24 00:00:00,1295648.0,0.0,0.0,4810,28008,2.0,0,0,3,4810,0,1990,2009,20418 NE 64th Pl,Redmond,WA 98053,USA


In [21]:
lpbed = df['sqft_living'][df['bedrooms']!=0].sum()/df['bedrooms'][df['bedrooms']!=0].sum()
lpbed

627.6491682309535

In [22]:
lpbath = df['sqft_living'][df['bathrooms']!=0].sum()/df['bathrooms'][df['bathrooms']!=0].sum()
lpbath

988.6874330869233

In [23]:
# mdf['bedrooms'] = df['bedrooms']
# mdf['bathrooms'] = df['bathrooms']

# mdf[mdf['bedrooms']==0]

# mdf['bedrooms'][mdf['bedrooms']==0] = df['sqft_living'][df['bedrooms']==0]/lpbed.astype(int).copy()
# mdf['bathrooms'][mdf['bathrooms']==0] = df['sqft_living'][df['bathrooms']==0]/lpbath.astype(int).copy()

#instead of chain indexing [][], make it loc indexing .loc[[],]

In [24]:
mdf.loc[mdf['bedrooms']==0, 'bedrooms'] = df.loc[mdf['bedrooms']==0, 'sqft_living'] // lpbed
mdf.loc[mdf['bathrooms']==0, 'bathrooms'] = df.loc[mdf['bathrooms']==0, 'sqft_living'] // lpbath

In [25]:
mdf[df['bedrooms']==0]

,price,month,bedrooms,bathrooms
2365,1095000,6,4.0,3.0
3209,1295648,6,7.0,4.0


In [26]:
mdf.iloc[[2365,3209]]

,price,month,bedrooms,bathrooms
2365,1095000,6,4.0,3.0
3209,1295648,6,7.0,4.0


In [27]:
df[df['bedrooms']==0]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
2365,2014-06-12 00:00:00,1095000.0,0.0,0.0,3064,4764,3.5,0,2,3,3064,0,1990,2009,814 E Howe St,Seattle,WA 98102,USA
3209,2014-06-24 00:00:00,1295648.0,0.0,0.0,4810,28008,2.0,0,0,3,4810,0,1990,2009,20418 NE 64th Pl,Redmond,WA 98053,USA


In [28]:
feature = 'sqft_living'
understand(feature,False)

In [29]:
# mdf.drop('sqft_living', axis=1, inplace=True)

In [30]:
feature = 'sqft_lot'
understand(feature,True)

In [31]:
# mdf['sqft_lot'] = df['sqft_lot']

In [32]:
feature = 'floors'
understand(feature,True)

In [33]:
feature = 'waterfront'
understand(feature,False)

In [34]:
feature = 'view'
understand(feature,False)

In [35]:
feature = 'condition'
understand(feature,False)

In [36]:
feature = 'sqft_above'
understand(feature,True)

In [37]:
feature = 'sqft_basement'
understand(feature,True)

In [38]:
feature = 'yr_built'
understand(feature,False)

In [39]:
mdf['Age1'] = 2014 - df['yr_built']

In [40]:
feature = 'yr_renovated'
understand(feature,False)

In [41]:
mdf['Age2'] = 2014 - df['yr_renovated']

In [42]:
# mdf['Age2'][df['yr_renovated']==0] = mdf['Age1']

In [43]:
mdf.loc[df['yr_renovated']==0,'Age2'] = mdf.loc[df['yr_renovated']==0,'Age1'].values

In [44]:
feature = 'street'
understand(feature,False)

In [45]:
feature = 'city'
understand(feature,False)

In [46]:
feature = 'statezip'
understand(feature,False)

In [47]:
from category_encoders import TargetEncoder

def one_hot(feature):
    fdf = pd.get_dummies(mdf, columns=[feature], prefix=[feature])

In [48]:
mdf

,price,month,bedrooms,bathrooms,sqft_lot,floors,sqft_above,sqft_basement,Age1,Age2
0,313000,5,3.0,1.50,7912,1.5,1340,0,59,9
1,2384000,5,5.0,2.50,9050,2.0,3370,280,93,93
2,342000,5,3.0,2.00,11947,1.0,1930,0,48,48
3,420000,5,3.0,2.25,8030,1.0,1000,1000,51,51
4,550000,5,4.0,2.50,10500,1.0,1140,800,38,22
...,...,...,...,...,...,...,...,...,...,...
4595,308166,7,3.0,1.75,6360,1.0,1510,0,60,35
4596,534333,7,3.0,2.50,7573,2.0,1460,0,31,5
4597,416904,7,3.0,2.50,7014,2.0,3010,0,5,5
4598,203400,7,4.0,2.00,6630,1.0,1070,1020,40,40


In [83]:
one_hot('month')
# one_hot('waterfront')
# one_hot('view')
# one_hot('condition')

In [50]:
fdf = mdf.copy()

In [51]:
fdf

,price,month,bedrooms,bathrooms,sqft_lot,floors,sqft_above,sqft_basement,Age1,Age2
0,313000,5,3.0,1.50,7912,1.5,1340,0,59,9
1,2384000,5,5.0,2.50,9050,2.0,3370,280,93,93
2,342000,5,3.0,2.00,11947,1.0,1930,0,48,48
3,420000,5,3.0,2.25,8030,1.0,1000,1000,51,51
4,550000,5,4.0,2.50,10500,1.0,1140,800,38,22
...,...,...,...,...,...,...,...,...,...,...
4595,308166,7,3.0,1.75,6360,1.0,1510,0,60,35
4596,534333,7,3.0,2.50,7573,2.0,1460,0,31,5
4597,416904,7,3.0,2.50,7014,2.0,3010,0,5,5
4598,203400,7,4.0,2.00,6630,1.0,1070,1020,40,40


In [52]:
dfarch = fdf.copy()

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [54]:
fdf[['sqft_lot', 'sqft_above', 'sqft_basement','Age1','Age2']] = scaler.fit_transform(fdf[['sqft_lot', 'sqft_above', 'sqft_basement','Age1','Age2']])

In [55]:
tar = fdf['price']

In [56]:
fdf2 = fdf.drop('price', axis=1)

In [57]:
fdf2

,month,bedrooms,bathrooms,sqft_lot,floors,sqft_above,sqft_basement,Age1,Age2
0,5,3.0,1.50,-0.192527,1.5,-0.564425,-0.671413,0.530825,-0.741524
1,5,5.0,2.50,-0.160880,2.0,1.811625,-0.065270,1.673421,2.386968
2,5,3.0,2.00,-0.080319,1.0,0.126151,-0.671413,0.161161,0.710990
3,5,3.0,2.25,-0.189245,1.0,-0.962384,1.493385,0.261979,0.822722
4,5,4.0,2.50,-0.120558,1.0,-0.798519,1.060425,-0.174896,-0.257352
...,...,...,...,...,...,...,...,...,...
4595,7,3.0,1.75,-0.235686,1.0,-0.365446,-0.671413,0.564431,0.226819
4596,7,3.0,2.50,-0.201954,2.0,-0.423969,-0.671413,-0.410136,-0.890499
4597,7,3.0,2.50,-0.217499,2.0,1.390257,-0.671413,-1.283886,-0.890499
4598,7,4.0,2.00,-0.228177,1.0,-0.880452,1.536681,-0.107685,0.413039


# Try models: LR,L1,L2 and RF, GB,XGB 

In [58]:
from sklearn.model_selection import train_test_split
X_trainvalid, X_test, y_trainvalid, y_test = train_test_split(fdf2, tar, test_size=0.1, random_state=1236)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainvalid, y_trainvalid, test_size=0.15, random_state=1236)
print(y_train.mean(),y_valid.mean(),y_test.mean())
print(len(y_train),len(y_valid),len(y_test))

547827.8735632184 560752.8796747967 533851.350877193
3480 615 456


In [59]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [60]:
y_train_hat = model.predict(X_train)
y_valid_hat = model.predict(X_valid)
y_test_hat = model.predict(X_test)

In [61]:
tar.describe().astype(int)

count       4551
mean      548174
std       355461
min         7800
25%       326050
50%       465000
75%       657500
max      4668000
Name: price, dtype: int32

In [62]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

In [63]:
print(mean_absolute_error(y_train,y_train_hat))
print(mean_absolute_error(y_valid,y_valid_hat))
print(mean_absolute_error(y_test,y_test_hat))

156692.97152394484
160751.28690425726
156617.2627541034


In [64]:
print(explained_variance_score(y_train,y_train_hat))
print(explained_variance_score(y_valid,y_valid_hat))
print(explained_variance_score(y_test,y_test_hat))

0.5420946470990576
0.5357880665635306
0.5046575998505445


In [65]:
# model.coef_

In [66]:
from sklearn.linear_model import Ridge
L2_model = Ridge(alpha=0.5)
L2_model.fit(X_train, y_train)

Ridge(alpha=0.5)

In [67]:
y_train_hat_L2 = L2_model.predict(X_train)
y_valid_hat_L2 = L2_model.predict(X_valid)
y_test_hat_L2 = L2_model.predict(X_test)

In [68]:
print(explained_variance_score(y_train,y_train_hat_L2))
print(explained_variance_score(y_valid,y_valid_hat_L2))
print(explained_variance_score(y_test,y_test_hat_L2))

0.5420946116211639
0.535769163216476
0.5046865689670468


In [69]:
from sklearn.linear_model import Lasso
L1_model = Lasso(alpha=0.1)
L1_model.fit(X_train, y_train)

Lasso(alpha=0.1)

In [70]:
y_train_hat_L1 = L1_model.predict(X_train)
y_valid_hat_L1 = L1_model.predict(X_valid)
y_test_hat_L1 = L1_model.predict(X_test)

In [71]:
print(explained_variance_score(y_train,y_train_hat_L1))
print(explained_variance_score(y_valid,y_valid_hat_L1))
print(explained_variance_score(y_test,y_test_hat_L1))

0.5420946470971445
0.5357880246784237
0.5046577837307524


In [72]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [73]:
y_train_hat_rf = rf_model.predict(X_train)
y_valid_hat_rf = rf_model.predict(X_valid)
y_test_hat_rf = rf_model.predict(X_test)

In [74]:
print(mean_absolute_error(y_train,y_train_hat_rf))
print(mean_absolute_error(y_valid,y_valid_hat_rf))
print(mean_absolute_error(y_test,y_test_hat_rf))

56845.60780842912
157361.87378319784
152562.92480994153


In [75]:
print(explained_variance_score(y_train,y_train_hat_rf))
print(explained_variance_score(y_valid,y_valid_hat_rf))
print(explained_variance_score(y_test,y_test_hat_rf))

0.9343376710155079
0.5249421264044607
0.4939033147600972


In [76]:
from sklearn.ensemble import GradientBoostingRegressor
GB_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
GB_model.fit(X_train, y_train)

GradientBoostingRegressor()

In [77]:
y_train_hat_GB = GB_model.predict(X_train)
y_valid_hat_GB = GB_model.predict(X_valid)
y_test_hat_GB = GB_model.predict(X_test)

In [78]:
print(explained_variance_score(y_train,y_train_hat_GB))
print(explained_variance_score(y_valid,y_valid_hat_GB))
print(explained_variance_score(y_test,y_test_hat_GB))

0.7132959898854855
0.5237320165245158
0.4906923352417307


In [79]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
xg_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05)
xg_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [80]:
y_train_hat_xg = xg_model.predict(X_train)
y_valid_hat_xg = xg_model.predict(X_valid)
y_test_hat_xg = xg_model.predict(X_test)

In [81]:
print(explained_variance_score(y_train,y_train_hat_xg))
print(explained_variance_score(y_valid,y_valid_hat_xg))
print(explained_variance_score(y_test,y_test_hat_xg))
# resplit the sample to give more records to valid but still way better train metric than valid or test
# this is a warning against overfitting, xgb is extremely good at fitting outliers at the expense of extream overfitting
# caution when picking xgb with small sample size

0.9736049243818069
0.4930249223219393
0.42191786024949585


In [82]:
print(mean_absolute_error(y_train,y_train_hat_xg))
print(mean_absolute_error(y_valid,y_valid_hat_xg))
print(mean_absolute_error(y_test,y_test_hat_xg))

40990.2883126796
160619.98224085366
153855.61485745615
